In [19]:
# %load /home/gaetan/Desktop/geovpylib/templates/heading.py
%load_ext autoreload
%autoreload 2

# Common imports
import os
import pandas as pd, numpy as np
import datetime
#import json
#import request
#import duckdb
#import plotly.express as px

# Geovpylib library
import geovpylib.analysis as a
import geovpylib.database as db
import geovpylib.decorators as d
import geovpylib.magics
import geovpylib.pks as pks
import geovpylib.queries as q
import geovpylib.sparql as sparql
import geovpylib.utils as u
eta = u.Eta()


env = 'prod'
pk_project = pks.projects.symogih
execute = True
metadata_str = 'collective-actor-correction'
import_manner = 'one-shot'

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Collective Actors Correction

## 1./ Delete everything

In [20]:
# Connect to Geovistory database
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

[DB] Connecting to PRODUCTION Database ... Connected!


### 1.0/ Fetch general data

In [3]:
existing_groups = db.query(f"""
    select 
        r1.pk_entity as pk_group,
        a4.string as uri
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_sameAsURI_URI}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = s2.fk_object_info and s3.fk_property = {pks.properties.appe_hasValue_string}
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    inner join information.appellation a4 on a4.pk_entity = s3.fk_object_info
    where r1.fk_class = {pks.classes.group}
""")
existing_groups['pk_bhp'] = [string[string.rfind('/') + 1:] for string in existing_groups['uri']]
existing_groups.drop(columns='uri', inplace=True)
existing_groups = existing_groups[existing_groups['pk_bhp'].str.contains('CoAc')]

pk_groups_str = u.get_sql_ready_str(existing_groups['pk_group'])

# a.infos(existing_groups)

#########

existing_formations = db.query(f"""
    select
        r1.pk_entity as pk_formation,
        ipr2.pk_entity as pk_ipr_has_formed_group
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = r1.pk_entity and s2.fk_property = {pks.properties.formation_hasFormed_group} and s2.fk_object_info in {pk_groups_str}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    where r1.fk_class = {pks.classes.formation}
""")
pk_formations_str = u.get_sql_ready_str(existing_formations['pk_formation'])

# a.infos(existing_formations)

#########

existing_dissolutions = db.query(f"""
    select
        r1.pk_entity as pk_dissolution,
        ipr2.pk_entity as pk_ipr_has_dissolved_group
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = r1.pk_entity and s2.fk_property = {pks.properties.dissolution_dissolved_group} and s2.fk_object_info in {pk_groups_str}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    where r1.fk_class = {pks.classes.dissolution}
""")
pk_dissolution_str = u.get_sql_ready_str(existing_dissolutions['pk_dissolution'])

# a.infos(existing_dissolutions)

#### 1.1.a/ Delete `has_formed_group` statement

In [4]:
pk_ipr_has_formed_group = u.get_sql_ready_str(existing_formations['pk_ipr_has_formed_group'])
db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_ipr_has_formed_group};
""")

#### 1.1.b/ Delete `has_dissolved_group` statement

In [5]:
pk_ipr_has_dissolved_group = u.get_sql_ready_str(existing_dissolutions['pk_ipr_has_dissolved_group'])
db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_ipr_has_dissolved_group};
""")

### 1.2/ Delete time information

In [6]:
time_properties = f"({pks.properties.timeSpan_atSomeTimeWithin_timePrimitive}, {pks.properties.timespan_beginOfTheBegin_timePrim}, {pks.properties.timespan_endOfTheEnd_timePrim}, {pks.properties.timeSpan_ongoingThroughout_timePrimitive}, {pks.properties.timespan_endOfTheBegin_timePrim}, {pks.properties.timespan_beginOfTheEnd_timePrim})"

#### 1.2.a/ Delete time information for formation

In [7]:
formation_time_stmts = db.query(f"""
    select
        ipr.pk_entity as pk_ipr_time_info_form
    from information.statement s
    inner join projects.info_proj_rel ipr on ipr.fk_entity = s.pk_entity and ipr.fk_project = {pk_project} and ipr.is_in_project = true
    where s.fk_subject_info in {pk_formations_str} and s.fk_property in {time_properties}
""")
pk_ipr_time_info_form = u.get_sql_ready_str(formation_time_stmts['pk_ipr_time_info_form'])

db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_ipr_time_info_form};
""")

#### 1.2.b/ Delete time information for dissolution

In [8]:
dissolution_time_stmts = db.query(f"""
    select
        ipr.pk_entity as pk_ipr_time_info_diss
    from information.statement s
    inner join projects.info_proj_rel ipr on ipr.fk_entity = s.pk_entity and ipr.fk_project = {pk_project} and ipr.is_in_project = true
    where s.fk_subject_info in {pk_dissolution_str} and s.fk_property in {time_properties}
""")
pk_ipr_time_info_diss = u.get_sql_ready_str(dissolution_time_stmts['pk_ipr_time_info_diss'])

db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_ipr_time_info_diss};
""")

### 1.3/ Delete certainty comment

#### 1.3.a/ Delete certainty comment for formations

In [9]:
formation_comments = db.query(f"""
    select
        r1.pk_entity as pk_comment,
        ipr1.pk_entity as pk_ipr_comment,
        ipr2.pk_entity as pk_ipr_stmt_has_comment,
        ipr3.pk_entity as pk_ipr_stmt_comment_has_type,
        ipr4.pk_entity as pk_ipr_stmt_comment_has_value
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_object_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_hasComment_text} and s2.fk_subject_info in {pk_formations_str}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = r1.pk_entity and s3.fk_property = {pks.properties.comment_hasCommentType_CommentType} and s3.fk_object_info = 7953586
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    inner join information.statement s4 on s4.fk_subject_info = r1.pk_entity and s4.fk_property = {pks.properties.text_hasValueVersion_string}
    inner join projects.info_proj_rel ipr4 on ipr4.fk_entity = s4.pk_entity and ipr4.fk_project = {pk_project} and ipr4.is_in_project = true
    where r1.fk_class = {pks.classes.comment}
""")

pk_iprs_cert_comment_form = formation_comments['pk_ipr_comment'].tolist() + formation_comments['pk_ipr_stmt_has_comment'].tolist() + formation_comments['pk_ipr_stmt_comment_has_type'].tolist() + formation_comments['pk_ipr_stmt_comment_has_value'].tolist()
pk_iprs_cert_comment_form = u.get_sql_ready_str(pk_iprs_cert_comment_form)

db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_iprs_cert_comment_form};
""")

#### 1.3.b/ Delete certainty comment for dissolutions

In [10]:
dissolution_comments = db.query(f"""
    select
        r1.pk_entity as pk_comment,
        ipr1.pk_entity as pk_ipr_comment,
        ipr2.pk_entity as pk_ipr_stmt_has_comment,
        ipr3.pk_entity as pk_ipr_stmt_comment_has_type,
        ipr4.pk_entity as pk_ipr_stmt_comment_has_value
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_object_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_hasComment_text} and s2.fk_subject_info in {pk_dissolution_str}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = r1.pk_entity and s3.fk_property = {pks.properties.comment_hasCommentType_CommentType} and s3.fk_object_info = 7953586
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    inner join information.statement s4 on s4.fk_subject_info = r1.pk_entity and s4.fk_property = {pks.properties.text_hasValueVersion_string}
    inner join projects.info_proj_rel ipr4 on ipr4.fk_entity = s4.pk_entity and ipr4.fk_project = {pk_project} and ipr4.is_in_project = true
    where r1.fk_class = {pks.classes.comment}
""")

pk_iprs_cert_comment_diss = dissolution_comments['pk_ipr_comment'].tolist() + dissolution_comments['pk_ipr_stmt_has_comment'].tolist() + dissolution_comments['pk_ipr_stmt_comment_has_type'].tolist() + dissolution_comments['pk_ipr_stmt_comment_has_value'].tolist()
pk_iprs_cert_comment_diss = u.get_sql_ready_str(pk_iprs_cert_comment_diss)

db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_iprs_cert_comment_diss};
""")

### 1.4/ Delete URI informations

#### 1.4.a/ Delete URI informations for formations

In [11]:
formation_uri = db.query(f"""
    select
        ipr1.pk_entity as pk_ipr_uri,
        ipr2.pk_entity as pk_ipr_stmt_same_as_uri,
        ipr3.pk_entity as pk_ipr_stmt_has_value
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_object_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_sameAsURI_URI} and s2.fk_subject_info in {pk_formations_str}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = r1.pk_entity and s3.fk_property = {pks.properties.appe_hasValue_string}
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    where r1.fk_class = {pks.classes.uri}
""")

pk_iprs_uri_form = formation_uri['pk_ipr_uri'].tolist() + formation_uri['pk_ipr_stmt_same_as_uri'].tolist() + formation_uri['pk_ipr_stmt_has_value'].tolist()
pk_iprs_uri_form = u.get_sql_ready_str(pk_iprs_uri_form)

db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_iprs_uri_form};
""")

#### 1.4.b/ Delete URI informations for dissolution

In [12]:
dissolution_uri = db.query(f"""
    select
        ipr1.pk_entity as pk_ipr_uri,
        ipr2.pk_entity as pk_ipr_stmt_same_as_uri,
        ipr3.pk_entity as pk_ipr_stmt_has_value
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_object_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_sameAsURI_URI} and s2.fk_subject_info in {pk_dissolution_str}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = r1.pk_entity and s3.fk_property = {pks.properties.appe_hasValue_string}
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    where r1.fk_class = {pks.classes.uri}
""")

pk_iprs_uri_diss = dissolution_uri['pk_ipr_uri'].tolist() + dissolution_uri['pk_ipr_stmt_same_as_uri'].tolist() + dissolution_uri['pk_ipr_stmt_has_value'].tolist()
pk_iprs_uri_diss = u.get_sql_ready_str(pk_iprs_uri_diss)

db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_iprs_uri_diss};
""")

### 1.5/ Delete date complements

#### 1.5.a/ Delete date complements on formations

In [13]:
formation_date_cmplt = db.query(f"""
    select
        r1.pk_entity as pk_comment,
        ipr1.pk_entity as pk_ipr_comment,
        ipr2.pk_entity as pk_ipr_stmt_has_comment,
        ipr3.pk_entity as pk_ipr_stmt_comment_has_type,
        ipr4.pk_entity as pk_ipr_stmt_comment_has_value
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_object_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_hasComment_text} and s2.fk_subject_info in {pk_formations_str}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = r1.pk_entity and s3.fk_property = {pks.properties.comment_hasCommentType_CommentType} and s3.fk_object_info = 8065621
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    inner join information.statement s4 on s4.fk_subject_info = r1.pk_entity and s4.fk_property = {pks.properties.text_hasValueVersion_string}
    inner join projects.info_proj_rel ipr4 on ipr4.fk_entity = s4.pk_entity and ipr4.fk_project = {pk_project} and ipr4.is_in_project = true
    where r1.fk_class = {pks.classes.comment}
""")

pk_iprs_cert_date_cmplt_form = formation_date_cmplt['pk_ipr_comment'].tolist() + formation_date_cmplt['pk_ipr_stmt_has_comment'].tolist() + formation_date_cmplt['pk_ipr_stmt_comment_has_type'].tolist() + formation_date_cmplt['pk_ipr_stmt_comment_has_value'].tolist()
pk_iprs_cert_date_cmplt_form = u.get_sql_ready_str(pk_iprs_cert_date_cmplt_form)

db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_iprs_cert_date_cmplt_form};
""")

#### 1.5.b/ Delete date complements on dissolution

In [18]:
dissolution_date_cmplt = db.query(f"""
    select
        r1.pk_entity as pk_comment,
        ipr1.pk_entity as pk_ipr_comment,
        ipr2.pk_entity as pk_ipr_stmt_has_comment,
        ipr3.pk_entity as pk_ipr_stmt_comment_has_type,
        ipr4.pk_entity as pk_ipr_stmt_comment_has_value
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_object_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_hasComment_text} and s2.fk_subject_info in {pk_dissolution_str}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = r1.pk_entity and s3.fk_property = {pks.properties.comment_hasCommentType_CommentType} and s3.fk_object_info = 8065621
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    inner join information.statement s4 on s4.fk_subject_info = r1.pk_entity and s4.fk_property = {pks.properties.text_hasValueVersion_string}
    inner join projects.info_proj_rel ipr4 on ipr4.fk_entity = s4.pk_entity and ipr4.fk_project = {pk_project} and ipr4.is_in_project = true
    where r1.fk_class = {pks.classes.comment}
""")

pk_iprs_cert_date_cmplt_diss = dissolution_date_cmplt['pk_ipr_comment'].tolist() + dissolution_date_cmplt['pk_ipr_stmt_has_comment'].tolist() + dissolution_date_cmplt['pk_ipr_stmt_comment_has_type'].tolist() + dissolution_date_cmplt['pk_ipr_stmt_comment_has_value'].tolist()
pk_iprs_cert_date_cmplt_diss = u.get_sql_ready_str(pk_iprs_cert_date_cmplt_diss)

if len(dissolution_date_cmplt) > 0:
    db.execute(f"""
        update projects.info_proj_rel 
            set is_in_project = false
            where pk_entity in {pk_iprs_cert_date_cmplt_diss};
    """)

### 1.6/ Delete note on dates

#### 1.6.a/ Delete note on dates on formations

In [21]:
formation_note_dates = db.query(f"""
    select
        r1.pk_entity as pk_comment,
        ipr1.pk_entity as pk_ipr_comment,
        ipr2.pk_entity as pk_ipr_stmt_has_comment,
        ipr3.pk_entity as pk_ipr_stmt_comment_has_type,
        ipr4.pk_entity as pk_ipr_stmt_comment_has_value
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_object_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_hasComment_text} and s2.fk_subject_info in {pk_formations_str}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = r1.pk_entity and s3.fk_property = {pks.properties.comment_hasCommentType_CommentType} and s3.fk_object_info = 8065632
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    inner join information.statement s4 on s4.fk_subject_info = r1.pk_entity and s4.fk_property = {pks.properties.text_hasValueVersion_string}
    inner join projects.info_proj_rel ipr4 on ipr4.fk_entity = s4.pk_entity and ipr4.fk_project = {pk_project} and ipr4.is_in_project = true
    where r1.fk_class = {pks.classes.comment}
""")

pk_iprs_cert_note_dates_form = formation_note_dates['pk_ipr_comment'].tolist() + formation_note_dates['pk_ipr_stmt_has_comment'].tolist() + formation_note_dates['pk_ipr_stmt_comment_has_type'].tolist() + formation_note_dates['pk_ipr_stmt_comment_has_value'].tolist()
pk_iprs_cert_note_dates_form = u.get_sql_ready_str(pk_iprs_cert_note_dates_form)

db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_iprs_cert_note_dates_form};
""")

#### 1.6.b/ Delete note on dates on dissolution

In [22]:
dissolution_note_dates = db.query(f"""
    select
        r1.pk_entity as pk_comment,
        ipr1.pk_entity as pk_ipr_comment,
        ipr2.pk_entity as pk_ipr_stmt_has_comment,
        ipr3.pk_entity as pk_ipr_stmt_comment_has_type,
        ipr4.pk_entity as pk_ipr_stmt_comment_has_value
    from information.resource r1
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_object_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_hasComment_text} and s2.fk_subject_info in {pk_dissolution_str}
    inner join projects.info_proj_rel ipr2 on ipr2.fk_entity = s2.pk_entity and ipr2.fk_project = {pk_project} and ipr2.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = r1.pk_entity and s3.fk_property = {pks.properties.comment_hasCommentType_CommentType} and s3.fk_object_info = 8065632
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    inner join information.statement s4 on s4.fk_subject_info = r1.pk_entity and s4.fk_property = {pks.properties.text_hasValueVersion_string}
    inner join projects.info_proj_rel ipr4 on ipr4.fk_entity = s4.pk_entity and ipr4.fk_project = {pk_project} and ipr4.is_in_project = true
    where r1.fk_class = {pks.classes.comment}
""")

pk_iprs_cert_note_dates_diss = dissolution_note_dates['pk_ipr_comment'].tolist() + dissolution_note_dates['pk_ipr_stmt_has_comment'].tolist() + dissolution_note_dates['pk_ipr_stmt_comment_has_type'].tolist() + dissolution_note_dates['pk_ipr_stmt_comment_has_value'].tolist()
pk_iprs_cert_note_dates_diss = u.get_sql_ready_str(pk_iprs_cert_note_dates_diss)

db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_iprs_cert_note_dates_diss};
""")

### 1.7/ Delete Formation, Dissolutions

In [23]:
db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_formations_str};
""")
db.execute(f"""
    update projects.info_proj_rel 
        set is_in_project = false
        where pk_entity in {pk_dissolution_str};
""")

## 2./ Find and create missing Groups

### 2.1/ Find all existing groups in symogih project in Geovistory

In [24]:
# Connect to Geovistory database
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

[DB] Connecting to PRODUCTION Database ... Connected!


In [25]:
# Find existing group on GV
groups = db.query(f"""
    select
        r1.pk_entity as pk_group,
        a4.string as uri
    from information.resource r1 
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_sameAsURI_URI}
    inner join projects.info_proj_rel ipr2a on ipr2a.fk_entity = s2.pk_entity and ipr2a.fk_project = {pk_project} and ipr2a.is_in_project = true
    inner join projects.info_proj_rel ipr2b on ipr2b.fk_entity = s2.fk_object_info and ipr2b.fk_project = {pk_project} and ipr2b.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = s2.fk_object_info and s3.fk_property = {pks.properties.appe_hasValue_string}
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    inner join information.appellation a4 on a4.pk_entity = s3.fk_object_info
    where r1.fk_class = {pks.classes.group}
""")
groups = groups[groups['uri'].str.contains('CoAc')]
groups['pk_coac'] = [string[string.rfind('/') + 1:] for string in groups['uri']]
groups.drop(columns='uri', inplace=True)

# a.infos(groups)

### 2.2/ Find all Collective actors from BHP (CSV file)

In [26]:
coac = u.read_df('../../data/bhp/collective-actor.csv', skip_info=True).rename(columns={'notes':'notes_coac', 'begin_year':'begin_year_coac', 'end_year':'end_year_coac'}).drop(columns=['concat_standard_name'])
coac_name = u.read_df('../../data/bhp/collective-actor-name.csv', skip_info=True).rename(columns={'notes':'notes_name', 'lang_iso':'lang_name', 'comment_begin_year':'comment_begin_year_name', 'comment_end_year':'comment_end_year_name', 'begin_date':'begin_date_name', 'end_date':'end_date_name'})
coac_text_property = u.read_df('../../data/bhp/collective-actor-text-property.csv', skip_info=True).rename(columns={'notes':'notes_text_prop', 'lang_iso_code':'lang_text_prop'})

coacs = coac.merge(coac_name, left_on='pk_collective_actor', right_on='fk_collective_actor', how='left').drop(columns=['fk_collective_actor', 'pk_collective_actor_name'])
coacs = coacs.merge(coac_text_property, left_on='pk_collective_actor', right_on='fk_collective_actor', how='left').drop(columns=['fk_collective_actor', 'pk_collective_actor_text_property'])
coacs['begin_date_name'] = [u.parse_tuple_date(d) for d in coacs['begin_date_name']]
coacs['end_date_name'] = [u.parse_tuple_date(d) for d in coacs['end_date_name']]

coacs['pk_collective_actor'] = 'CoAc' + coacs['pk_collective_actor'].astype(str)
coacs.rename(columns={'pk_collective_actor':'pk_coac'}, inplace=True)

# a.infos(coacs)

### 2.3/ Get record linkage result (CSV file)

In [27]:
record_linkage = pd.read_csv('../../data/record-linkage-collective-actors-filled.csv')
record_linkage = record_linkage[record_linkage['Doublon'] == 'oui']
record_linkage = record_linkage[['pk_bhp', 'pk_gv']]
record_linkage['pk_bhp'] = 'CoAc' + record_linkage['pk_bhp'].astype(str)
record_linkage.rename(columns={'pk_bhp': 'pk_coac', 'pk_gv': 'pk_group'}, inplace=True)

# a.infos(record_linkage)

### 2.4/ Find missing ones

In [28]:
existing_groups = groups['pk_coac'].tolist()
record_linkage_groups = record_linkage['pk_coac'].tolist()
missing_groups = coacs[[pk_coac not in existing_groups and pk_coac not in record_linkage_groups for pk_coac in coacs['pk_coac']]].reset_index(drop=True)

# All missing ones should be either existing or in the record linkage
assert len(missing_groups) == 0

### 2.5/ Add record linkage result to project

The problem was actually that all information about the entities where added, but not the entity itself.. Dumb me..

In [29]:
db.info_proj_rels.create(record_linkage['pk_group'])

Creating info_proj_rel of 132 entities with project <6857901> ... Done in [00h00m00s]


## 3./ Add Formations

### 3.1/ Fetch local file about coacs

In [76]:
coac = u.read_df('../../data/bhp/collective-actor.csv', skip_info=True).rename(columns={'notes':'notes_coac', 'begin_year':'begin_year_coac', 'end_year':'end_year_coac'}).drop(columns=['concat_standard_name'])
coac_name = u.read_df('../../data/bhp/collective-actor-name.csv', skip_info=True).rename(columns={'notes':'notes_name', 'lang_iso':'lang_name', 'comment_begin_year':'comment_begin_year_name', 'comment_end_year':'comment_end_year_name', 'begin_date':'begin_date_name', 'end_date':'end_date_name'})
coac_text_property = u.read_df('../../data/bhp/collective-actor-text-property.csv', skip_info=True).rename(columns={'notes':'notes_text_prop', 'lang_iso_code':'lang_text_prop'})

coacs = coac.merge(coac_name, left_on='pk_collective_actor', right_on='fk_collective_actor', how='left').drop(columns=['fk_collective_actor', 'pk_collective_actor_name'])
coacs = coacs.merge(coac_text_property, left_on='pk_collective_actor', right_on='fk_collective_actor', how='left').drop(columns=['fk_collective_actor', 'pk_collective_actor_text_property'])
coacs['begin_date_name'] = [u.parse_tuple_date(d) for d in coacs['begin_date_name']]
coacs['end_date_name'] = [u.parse_tuple_date(d) for d in coacs['end_date_name']]

coacs['pk_collective_actor'] = 'CoAc' + coacs['pk_collective_actor'].astype(str)
coacs.rename(columns={'pk_collective_actor':'pk_coac'}, inplace=True)

# a.infos(coacs)

### 3.2/ Fetch Geovistory existing groups

In [78]:
# Connect to Geovistory database
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

[DB] Connecting to PRODUCTION Database ... Connected!


In [79]:
# Find existing group on GV
groups = db.query(f"""
    select
        r1.pk_entity as pk_group,
        a4.string as uri
    from information.resource r1 
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_sameAsURI_URI}
    inner join projects.info_proj_rel ipr2a on ipr2a.fk_entity = s2.pk_entity and ipr2a.fk_project = {pk_project} and ipr2a.is_in_project = true
    inner join projects.info_proj_rel ipr2b on ipr2b.fk_entity = s2.fk_object_info and ipr2b.fk_project = {pk_project} and ipr2b.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = s2.fk_object_info and s3.fk_property = {pks.properties.appe_hasValue_string}
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    inner join information.appellation a4 on a4.pk_entity = s3.fk_object_info
    where r1.fk_class = {pks.classes.group}
""")
groups = groups[groups['uri'].str.contains('CoAc')]
groups['pk_coac'] = [string[string.rfind('/') + 1:] for string in groups['uri']]
groups.drop(columns='uri', inplace=True)

coacs_str = u.get_sql_ready_str(groups['pk_coac'])

# a.infos(groups)


### 3.3/ Fetch BHP related formation

In [80]:
# Connect to BHP database
db_url_env_var_name = 'YELLOW_BHP' # Name of an environment variable holding the Postgres database URL
db.connect_external(os.getenv(db_url_env_var_name), execute=False)

[DB] Requests will not be executed
[DB] Connecting to PGSQL Database ... Connected!


In [81]:
formations_bhp_info = db.query(f"""
    select
        ir.fk_associated_object as pk_coac, 
        i.pk_information, 
        id.year, id.month, id.day,
        id.fk_abob_type_information_date,
        id.complement as complement, 
        id.notes as notes,
        id.certainty_date
    from bhp.information_role ir
    inner join bhp.information i on i.pk_information = ir.fk_information and i.fk_type_information = 30
    inner join bhp.information_date id on id.fk_information = i.pk_information
    where ir.fk_associated_object in {coacs_str} and ir.fk_type_role = 49
""")

formations_bhp_info['year'] = formations_bhp_info['year'].astype(pd.Int64Dtype())
formations_bhp_info['month'] = formations_bhp_info['month'].astype(pd.Int64Dtype())
formations_bhp_info['day'] = formations_bhp_info['day'].astype(pd.Int64Dtype())
formations_bhp_info['fk_abob_type_information_date'] = formations_bhp_info['fk_abob_type_information_date'].astype(pd.Int64Dtype())
formations_bhp_info['date_bhp'] = [(row.year, row.month, row.day) for i, row in formations_bhp_info.iterrows()]
formations_bhp_info['uri'] = ['http://symogih.org/resource/Info' + str(fk_info) for fk_info in formations_bhp_info['pk_information']]
formations_bhp_info.drop(columns=['year', 'month', 'day', 'pk_information'], inplace=True)
formations_bhp_info['complement'] = [pd.NA if pd.isna(row['complement']) or row['complement'].strip() == '' else row['complement'] for _,row in formations_bhp_info.iterrows()]
formations_bhp_info['notes'] = [pd.NA if pd.isna(row['notes']) or row['notes'].strip() == '' else row['notes'] for _,row in formations_bhp_info.iterrows()]
formations_bhp_info['notes'] = [s.replace('<p>', '').replace('</p>', '') if pd.notna(s) else pd.NA for s in formations_bhp_info['notes']]
formations_bhp_info['notes'] = [s.replace('<em>', '').replace('</em>', '') if pd.notna(s) else pd.NA for s in formations_bhp_info['notes']]
formations_bhp_info['complement'] = [e.replace('<p>', '').replace('</p>', '') if pd.notna(e) else pd.NA for e in formations_bhp_info['complement']]

# a.infos(formations_bhp_info)

### 3.4/ Build import table

#### 3.4.1/ Create table, and link with GV id

In [82]:
formations = pd.DataFrame()
formations['pk_coac'] = np.unique(coacs['pk_coac'].tolist() + formations_bhp_info['pk_coac'].tolist())
formations = formations.merge(groups)

formations.sort_values('pk_coac', inplace=True)
formations = formations[['pk_group', 'pk_coac']]

# a.infos(formations)

#### 3.4.2/ Add URI

In [83]:
# Only concerns the BHP informations

formations['uri'] = pd.NA
formations.set_index('pk_coac', inplace=True)

for _, row in formations_bhp_info.iterrows():
    formations.at[row['pk_coac'], 'uri'] = row['uri']

formations.reset_index(inplace=True)

# a.infos(formations)

#### 3.4.3/ Add date informations

In [84]:
### Begin of the begin ###
note_begin = 4
property_name = 'begin_of_the_begin'
abob_types = [1125,1321,1322]

formations[property_name] = pd.NA
formations.set_index('pk_coac', inplace=True)

# From CSV file
selection = coacs[coacs['notes_begin'] == note_begin][['pk_coac', 'begin_year_coac']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.notna(row['begin_year_coac']) and pd.isna(formations.at[row['pk_coac'], 'uri']):
            formations.at[row['pk_coac'], property_name] = (row['begin_year_coac'], pd.NA, pd.NA)

# From BHP infomation (has prio over CSV)
selection = formations_bhp_info[formations_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        formations.at[row['pk_coac'], property_name] = row['date_bhp']

formations.reset_index(inplace=True)

# a.infos(formations)

In [85]:
### Begin of the end ###
property_name = 'begin_of_the_end'
abob_types = [1290]

formations[property_name] = pd.NA
formations.set_index('pk_coac', inplace=True)

# From BHP infomation (has prio over CSV)
selection = formations_bhp_info[formations_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.isna(formations.at[row['pk_coac'], 'uri']):
            formations.at[row['pk_coac'], property_name] = row['date_bhp']

formations.reset_index(inplace=True)

# a.infos(formations)

In [86]:
### End of the begin ###
property_name = 'end_of_the_begin'
abob_types = [1323]

formations[property_name] = pd.NA
formations.set_index('pk_coac', inplace=True)

# From BHP infomation (has prio over CSV)
selection = formations_bhp_info[formations_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.isna(formations.at[row['pk_coac'], 'uri']):
            formations.at[row['pk_coac'], property_name] = row['date_bhp']

formations.reset_index(inplace=True)

# a.infos(formations)

In [87]:
### End of the end ###
note_begin = 1
property_name = 'end_of_the_end'
abob_types = [256,1126,1128]

formations[property_name] = pd.NA
formations.set_index('pk_coac', inplace=True)

# From CSV file
selection = coacs[coacs['notes_begin'] == note_begin][['pk_coac', 'begin_year_coac']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.notna(row['begin_year_coac']) and pd.isna(formations.at[row['pk_coac'], 'uri']):
            formations.at[row['pk_coac'], property_name] = (row['begin_year_coac'], pd.NA, pd.NA)

# From BHP infomation (has prio over CSV)
selection = formations_bhp_info[formations_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        formations.at[row['pk_coac'], property_name] = row['date_bhp']

formations.reset_index(inplace=True)

# a.infos(formations)

In [88]:
### Ongoing throughout ###
note_begin = 3
property_name = 'ongoing_throughout'
abob_types = [258]

formations[property_name] = pd.NA
formations.set_index('pk_coac', inplace=True)

# From CSV file
selection = coacs[coacs['notes_begin'] == note_begin][['pk_coac', 'begin_year_coac']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.notna(row['begin_year_coac']) and pd.isna(formations.at[row['pk_coac'], 'uri']):
            formations.at[row['pk_coac'], property_name] = (row['begin_year_coac'], pd.NA, pd.NA)

# From BHP infomation (has prio over CSV)
selection = formations_bhp_info[formations_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        formations.at[row['pk_coac'], property_name] = row['date_bhp']

formations.reset_index(inplace=True)

# a.infos(formations)

In [89]:
### At some time within ###
note_begin = 2
property_name = 'at_some_time_within'
abob_types = [246]

formations[property_name] = pd.NA
formations.set_index('pk_coac', inplace=True)

# From CSV file
selection = coacs[coacs['notes_begin'] == note_begin][['pk_coac', 'begin_year_coac']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.notna(row['begin_year_coac']) and pd.isna(formations.at[row['pk_coac'], 'uri']):
            formations.at[row['pk_coac'], property_name] = (row['begin_year_coac'], pd.NA, pd.NA)

# From BHP infomation (has prio over CSV)
selection = formations_bhp_info[formations_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        formations.at[row['pk_coac'], property_name] = row['date_bhp']

formations.reset_index(inplace=True)

# a.infos(formations)

In [93]:
# Default cases: At some time within

formations.set_index('pk_coac', inplace=True)

# From CSV file
for i, row in coacs.iterrows():
    if pd.isna(row['notes_begin']) or row['notes_begin'] not in [1,2,3,4]:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            if pd.notna(row['begin_year_coac']) and pd.isna(formations.at[row['pk_coac'], 'uri']):
                formations.at[row['pk_coac'], 'at_some_time_within'] = (row['begin_year_coac'], pd.NA, pd.NA)

# From BHP infos
for i, row in formations_bhp_info.iterrows():
    if pd.isna(row['fk_abob_type_information_date']) or row['fk_abob_type_information_date'] not in [246,1125,1126,258,1289,1290,1321,1322,1323,256,1128,1128]:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            if pd.notna(row['date_bhp']) and pd.notna(row['date_bhp'][0]) and pd.notna(row['date_bhp'][1]) and pd.notna(row['date_bhp'][2]):
                formations.at[row['pk_coac'], 'at_some_time_within'] = row['date_bhp']        

formations.reset_index(inplace=True)

# a.infos(formations)

#### 3.4.4/ Add certainty comment

In [94]:
formations['certainty_comment'] = pd.NA

formations.set_index('pk_coac', inplace=True)

# From CSV file
for _, row in coacs.iterrows():
    if pd.notna(row['certainty_begin']) and row['certainty_begin'] == 2:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            formations.at[row['pk_coac'], 'certainty_comment'] = "Date reconstituée"
    elif pd.notna(row['certainty_begin']) and row['certainty_begin'] == 3:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            formations.at[row['pk_coac'], 'certainty_comment'] = "Date postulée"

# From BHP info
for _, row in formations_bhp_info.iterrows():
    if pd.notna(row['certainty_date']) and row['certainty_date'] == 2:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            formations.at[row['pk_coac'], 'certainty_comment'] = "Date reconstituée"
    elif pd.notna(row['certainty_date']) and row['certainty_date'] == 3:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            formations.at[row['pk_coac'], 'certainty_comment'] = "Date postulée"

formations.reset_index(inplace=True)

# a.infos(formations)

#### 3.4.5/ Add Date complement

In [95]:
formations['date_complement'] = pd.NA

formations.set_index('pk_coac', inplace=True)

# From BHP info
for _, row in formations_bhp_info.iterrows():
    formations.at[row['pk_coac'], 'date_complement'] = row['complement']
    
formations.reset_index(inplace=True)

# a.infos(formations)

#### 3.4.6/ Add date note

In [96]:
formations['date_note'] = pd.NA

formations.set_index('pk_coac', inplace=True)

# From BHP info
for _, row in formations_bhp_info.iterrows():
    formations.at[row['pk_coac'], 'date_note'] = '[Note] ' + row['notes']
    
formations.reset_index(inplace=True)

# a.infos(formations)

### 3.5/ Remove formation that should not be created

In [97]:
formations = formations[
    pd.notna(formations['begin_of_the_begin']) |
    pd.notna(formations['begin_of_the_end']) |
    pd.notna(formations['end_of_the_begin']) |
    pd.notna(formations['end_of_the_end']) |
    pd.notna(formations['ongoing_throughout']) |
    pd.notna(formations['at_some_time_within']) |
    pd.notna(formations['uri']) |
    pd.notna(formations['certainty_comment']) |
    pd.notna(formations['date_complement']) |
    pd.notna(formations['date_note'])
].reset_index(drop=True)

# a.infos(formations)

### 3.6/ Generate CSV for validation

In [100]:
formations.to_csv('./formation.csv', index=False)

a.infos(formations)

Shape:  (10833, 12) - extract:


,pk_coac,pk_group,uri,begin_of_the_begin,begin_of_the_end,end_of_the_begin,end_of_the_end,ongoing_throughout,at_some_time_within,certainty_comment,date_complement,date_note
0,CoAc10000,8366930,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"(1926, <NA>, <NA>)",<NA>,<NA>,<NA>
1,CoAc10003,8366703,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"(1907, <NA>, <NA>)",<NA>,<NA>,<NA>
2,CoAc10007,8366858,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"(1854, <NA>, <NA>)",<NA>,<NA>,<NA>
3,CoAc10008,8366583,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"(1896, <NA>, <NA>)",<NA>,<NA>,<NA>
4,CoAc10009,8366690,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"(1944, <NA>, <NA>)",<NA>,<NA>,<NA>


### 3.7/ Date validation

In [101]:
dates_prop = ['begin_of_the_begin', 'begin_of_the_begin', 'end_of_the_begin', 'end_of_the_end', 'ongoing_throughout', 'at_some_time_within']
min_year = float('inf')
max_year = float('-inf')
for i, row in formations.iterrows():
    for prop in dates_prop:
        if pd.isna(row[prop]): continue

        year = row[prop][0]
        month = row[prop][1]
        day = row[prop][2]
        
        is_pb = False

        # Year checkings
        if pd.notna(year):
            if year < min_year: min_year = year
            if year > max_year: max_year = year

        # Month checkings
        if pd.notna(month):
            if month != round(month): is_pb = True
            if month < 1 or month > 12: is_pb = True

        # Day checkings
        if pd.notna(day):    
            if day != round(day): is_pb = True
            if day < 1 or day > 31: is_pb = True

        if is_pb:
            print('line:', i, row['pk_coac'], prop, row[prop], year, month, day)

print('Year range:', min_year, '-', max_year)

line: 443 CoAc11679 at_some_time_within (1819, 0, 19) 1819 0 19
line: 9806 CoAc228 at_some_time_within (1827, 99, <NA>) 1827 99 <NA>
Year range: 0 - 2019


In [102]:
# Manual editing
formations.at[439, 'at_some_time_within'] = (1819, pd.NA, pd.NA)
formations.at[9806, 'at_some_time_within'] = (1827, pd.NA, pd.NA)   

## 4./ Add Dissolutions

### 4.1/ Fetch local file about coacs

In [103]:
coac = u.read_df('../../data/bhp/collective-actor.csv', skip_info=True).rename(columns={'notes':'notes_coac', 'begin_year':'begin_year_coac', 'end_year':'end_year_coac'}).drop(columns=['concat_standard_name'])
coac_name = u.read_df('../../data/bhp/collective-actor-name.csv', skip_info=True).rename(columns={'notes':'notes_name', 'lang_iso':'lang_name', 'comment_begin_year':'comment_begin_year_name', 'comment_end_year':'comment_end_year_name', 'begin_date':'begin_date_name', 'end_date':'end_date_name'})
coac_text_property = u.read_df('../../data/bhp/collective-actor-text-property.csv', skip_info=True).rename(columns={'notes':'notes_text_prop', 'lang_iso_code':'lang_text_prop'})

coacs = coac.merge(coac_name, left_on='pk_collective_actor', right_on='fk_collective_actor', how='left').drop(columns=['fk_collective_actor', 'pk_collective_actor_name'])
coacs = coacs.merge(coac_text_property, left_on='pk_collective_actor', right_on='fk_collective_actor', how='left').drop(columns=['fk_collective_actor', 'pk_collective_actor_text_property'])
coacs['begin_date_name'] = [u.parse_tuple_date(d) for d in coacs['begin_date_name']]
coacs['end_date_name'] = [u.parse_tuple_date(d) for d in coacs['end_date_name']]

coacs['pk_collective_actor'] = 'CoAc' + coacs['pk_collective_actor'].astype(str)
coacs.rename(columns={'pk_collective_actor':'pk_coac'}, inplace=True)

# a.infos(coacs)

### 4.2/ Fetch Geovistory existing groups

In [104]:
# Connect to Geovistory database
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

[DB] Connecting to PRODUCTION Database ... Connected!


In [105]:
# Find existing group on GV
groups = db.query(f"""
    select
        r1.pk_entity as pk_group,
        a4.string as uri
    from information.resource r1 
    inner join projects.info_proj_rel ipr1 on ipr1.fk_entity = r1.pk_entity and ipr1.fk_project = {pk_project} and ipr1.is_in_project = true
    inner join information.statement s2 on s2.fk_subject_info = r1.pk_entity and s2.fk_property = {pks.properties.entity_sameAsURI_URI}
    inner join projects.info_proj_rel ipr2a on ipr2a.fk_entity = s2.pk_entity and ipr2a.fk_project = {pk_project} and ipr2a.is_in_project = true
    inner join projects.info_proj_rel ipr2b on ipr2b.fk_entity = s2.fk_object_info and ipr2b.fk_project = {pk_project} and ipr2b.is_in_project = true
    inner join information.statement s3 on s3.fk_subject_info = s2.fk_object_info and s3.fk_property = {pks.properties.appe_hasValue_string}
    inner join projects.info_proj_rel ipr3 on ipr3.fk_entity = s3.pk_entity and ipr3.fk_project = {pk_project} and ipr3.is_in_project = true
    inner join information.appellation a4 on a4.pk_entity = s3.fk_object_info
    where r1.fk_class = {pks.classes.group}
""")
groups = groups[groups['uri'].str.contains('CoAc')]
groups['pk_coac'] = [string[string.rfind('/') + 1:] for string in groups['uri']]
groups.drop(columns='uri', inplace=True)

coacs_str = u.get_sql_ready_str(groups['pk_coac'])

# a.infos(groups)


### 4.3/ Fetch BHP related formation

In [106]:
# Connect to BHP database
db_url_env_var_name = 'YELLOW_BHP' # Name of an environment variable holding the Postgres database URL
db.connect_external(os.getenv(db_url_env_var_name), execute=False)

[DB] Requests will not be executed
[DB] Connecting to PGSQL Database ... Connected!


In [107]:
dissolutions_bhp_info = db.query(f"""
    select
        ir.fk_associated_object as pk_coac, 
        i.pk_information, 
        id.year, id.month, id.day,
        id.fk_abob_type_information_date,
        id.complement as complement, 
        id.notes as notes,
        id.certainty_date
    from bhp.information_role ir
    inner join bhp.information i on i.pk_information = ir.fk_information and i.fk_type_information = 33
    inner join bhp.information_date id on id.fk_information = i.pk_information
    where ir.fk_associated_object in {coacs_str} and ir.fk_type_role = 54
""")

dissolutions_bhp_info['year'] = dissolutions_bhp_info['year'].astype(pd.Int64Dtype())
dissolutions_bhp_info['month'] = dissolutions_bhp_info['month'].astype(pd.Int64Dtype())
dissolutions_bhp_info['day'] = dissolutions_bhp_info['day'].astype(pd.Int64Dtype())
dissolutions_bhp_info['fk_abob_type_information_date'] = dissolutions_bhp_info['fk_abob_type_information_date'].astype(pd.Int64Dtype())
dissolutions_bhp_info['date_bhp'] = [(row.year, row.month, row.day) for i, row in dissolutions_bhp_info.iterrows()]
dissolutions_bhp_info['uri'] = ['http://symogih.org/resource/Info' + str(fk_info) for fk_info in dissolutions_bhp_info['pk_information']]
dissolutions_bhp_info.drop(columns=['year', 'month', 'day', 'pk_information'], inplace=True)
dissolutions_bhp_info['complement'] = [pd.NA if pd.isna(row['complement']) or row['complement'].strip() == '' else row['complement'] for _,row in dissolutions_bhp_info.iterrows()]
dissolutions_bhp_info['notes'] = [pd.NA if pd.isna(row['notes']) or row['notes'].strip() == '' else row['notes'] for _,row in dissolutions_bhp_info.iterrows()]
dissolutions_bhp_info['complement'] = [e.replace('<p>', '').replace('</p>', '') if pd.notna(e) else pd.NA for e in dissolutions_bhp_info['complement']]

a.infos(dissolutions_bhp_info)

Shape:  (51, 7) - extract:


,pk_coac,fk_abob_type_information_date,complement,notes,certainty_date,date_bhp,uri
0,CoAc225,246,<NA>,<NA>,1,"(1847, 4, 30)",http://symogih.org/resource/Info41791
1,CoAc15408,246,<NA>,<p>Non documenté après 1935</p>,3,"(1935, <NA>, <NA>)",http://symogih.org/resource/Info112695
2,CoAc9137,246,<NA>,<NA>,1,"(1934, <NA>, <NA>)",http://symogih.org/resource/Info45645
3,CoAc227,246,<NA>,<NA>,1,"(1882, 1, <NA>)",http://symogih.org/resource/Info41902
4,CoAc231,246,<NA>,<NA>,1,"(1852, 6, 30)",http://symogih.org/resource/Info42155


### 4.4/ Build import table

#### 4.4.1/ Create table, and link with GV id

In [108]:
dissolutions = pd.DataFrame()
dissolutions['pk_coac'] = np.unique(coacs['pk_coac'].tolist() + dissolutions_bhp_info['pk_coac'].tolist())
dissolutions = dissolutions.merge(groups)

dissolutions.sort_values('pk_coac', inplace=True)
dissolutions = dissolutions[['pk_group', 'pk_coac']]

# a.infos(dissolutions)

#### 4.4.2/ Add URI

In [109]:
# Only concerns the BHP informations

dissolutions['uri'] = pd.NA
dissolutions.set_index('pk_coac', inplace=True)

for _, row in dissolutions_bhp_info.iterrows():
    dissolutions.at[row['pk_coac'], 'uri'] = row['uri']

dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

#### 4.4.3/ Add date informations

In [110]:
### Begin of the begin ###
note_end = 4
property_name = 'begin_of_the_begin'
abob_types = [1125,1321,1322]

dissolutions[property_name] = pd.NA
dissolutions.set_index('pk_coac', inplace=True)

# From CSV file
selection = coacs[coacs['notes_end'] == note_end][['pk_coac', 'end_year_coac']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.notna(row['end_year_coac']) and pd.isna(dissolutions.at[row['pk_coac'], 'uri']):
            dissolutions.at[row['pk_coac'], property_name] = (row['end_year_coac'], pd.NA, pd.NA)

# From BHP infomation (has prio over CSV)
selection = dissolutions_bhp_info[dissolutions_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        dissolutions.at[row['pk_coac'], property_name] = row['date_bhp']

dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

In [111]:
### Begin of the end ###
property_name = 'begin_of_the_end'
abob_types = [1290]

dissolutions[property_name] = pd.NA
dissolutions.set_index('pk_coac', inplace=True)

# From BHP infomation (has prio over CSV)
selection = dissolutions_bhp_info[dissolutions_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.isna(dissolutions.at[row['pk_coac'], 'uri']):
            dissolutions.at[row['pk_coac'], property_name] = row['date_bhp']

dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

In [112]:
### End of the begin ###
property_name = 'end_of_the_begin'
abob_types = [1323]

dissolutions[property_name] = pd.NA
dissolutions.set_index('pk_coac', inplace=True)

# From BHP infomation (has prio over CSV)
selection = dissolutions_bhp_info[dissolutions_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.isna(dissolutions.at[row['pk_coac'], 'uri']):
            dissolutions.at[row['pk_coac'], property_name] = row['date_bhp']

dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

In [113]:
### End of the end ###
note_end = 1
property_name = 'end_of_the_end'
abob_types = [256,1126,1128]

dissolutions[property_name] = pd.NA
dissolutions.set_index('pk_coac', inplace=True)

# From CSV file
selection = coacs[coacs['notes_end'] == note_end][['pk_coac', 'end_year_coac']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.notna(row['end_year_coac']) and pd.isna(dissolutions.at[row['pk_coac'], 'uri']):
            dissolutions.at[row['pk_coac'], property_name] = (row['end_year_coac'], pd.NA, pd.NA)

# From BHP infomation (has prio over CSV)
selection = dissolutions_bhp_info[dissolutions_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        dissolutions.at[row['pk_coac'], property_name] = row['date_bhp']

dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

In [114]:
### Ongoing throughout ###
note_end = 3
property_name = 'ongoing_throughout'
abob_types = [258]

dissolutions[property_name] = pd.NA
dissolutions.set_index('pk_coac', inplace=True)

# From CSV file
selection = coacs[coacs['notes_end'] == note_end][['pk_coac', 'end_year_coac']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.notna(row['end_year_coac']) and pd.isna(dissolutions.at[row['pk_coac'], 'uri']):
            dissolutions.at[row['pk_coac'], property_name] = (row['end_year_coac'], pd.NA, pd.NA)

# From BHP infomation (has prio over CSV)
selection = dissolutions_bhp_info[dissolutions_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        dissolutions.at[row['pk_coac'], property_name] = row['date_bhp']

dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

In [115]:
### At some time within ###
note_end = 2
property_name = 'at_some_time_within'
abob_types = [246]

dissolutions[property_name] = pd.NA
dissolutions.set_index('pk_coac', inplace=True)

# From CSV file
selection = coacs[coacs['notes_end'] == note_end][['pk_coac', 'end_year_coac']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        if pd.notna(row['end_year_coac']) and pd.isna(dissolutions.at[row['pk_coac'], 'uri']):
            dissolutions.at[row['pk_coac'], property_name] = (row['end_year_coac'], pd.NA, pd.NA)

# From BHP infomation (has prio over CSV)
selection = dissolutions_bhp_info[dissolutions_bhp_info['fk_abob_type_information_date'].isin(abob_types)][['pk_coac', 'date_bhp']]
for _, row in selection.iterrows():
    if row['pk_coac'] in groups['pk_coac'].tolist():
        dissolutions.at[row['pk_coac'], property_name] = row['date_bhp']

dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

In [116]:
# Default cases: At some time within

dissolutions.set_index('pk_coac', inplace=True)

# From CSV file
for i, row in coacs.iterrows():
    if pd.isna(row['notes_end']) or row['notes_end'] not in [1,2,3,4]:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            if pd.notna(row['end_year_coac']) and pd.isna(dissolutions.at[row['pk_coac'], 'uri']):
                dissolutions.at[row['pk_coac'], 'at_some_time_within'] = (row['end_year_coac'], pd.NA, pd.NA)

# From BHP infos
for i, row in dissolutions_bhp_info.iterrows():
    if row['fk_abob_type_information_date'] not in [246,1125,1126,258,1289,1290,1321,1322,1323,256,1128,1128]:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            if row['date_bhp'] != (pd.NA, pd.NA, pd.NA):
                dissolutions.at[row['pk_coac'], 'at_some_time_within'] = row['date_bhp']        

dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

#### 4.4.4/ Add certainty comment

In [117]:
dissolutions['certainty_comment'] = pd.NA

dissolutions.set_index('pk_coac', inplace=True)

# From CSV file
for _, row in coacs.iterrows():
    if pd.notna(row['certainty_end']) and row['certainty_end'] == 2:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            dissolutions.at[row['pk_coac'], 'certainty_comment'] = "Date reconstituée"
    elif pd.notna(row['certainty_end']) and row['certainty_end'] == 3:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            dissolutions.at[row['pk_coac'], 'certainty_comment'] = "Date postulée"

# From BHP info
for _, row in dissolutions_bhp_info.iterrows():
    if pd.notna(row['certainty_date']) and row['certainty_date'] == 2:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            dissolutions.at[row['pk_coac'], 'certainty_comment'] = "Date reconstituée"
    elif pd.notna(row['certainty_date']) and row['certainty_date'] == 3:
        if row['pk_coac'] in groups['pk_coac'].tolist():
            dissolutions.at[row['pk_coac'], 'certainty_comment'] = "Date postulée"

dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

#### 4.4.5/ Add Date complement

In [118]:
dissolutions['date_complement'] = pd.NA

dissolutions.set_index('pk_coac', inplace=True)

# From BHP info
for _, row in dissolutions_bhp_info.iterrows():
    dissolutions.at[row['pk_coac'], 'date_complement'] = row['complement']
    
dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

#### 4.4.6/ Add date note

In [119]:
dissolutions['date_note'] = pd.NA

dissolutions.set_index('pk_coac', inplace=True)

# From BHP info
for _, row in dissolutions_bhp_info.iterrows():
    dissolutions.at[row['pk_coac'], 'date_note'] = '[Note] ' + row['notes']
    
dissolutions.reset_index(inplace=True)

# a.infos(dissolutions)

### 4.5/ Remove dissolutions that should not be created

In [120]:
dissolutions = dissolutions[
    pd.notna(dissolutions['begin_of_the_begin']) |
    pd.notna(dissolutions['begin_of_the_end']) |
    pd.notna(dissolutions['end_of_the_begin']) |
    pd.notna(dissolutions['end_of_the_end']) |
    pd.notna(dissolutions['ongoing_throughout']) |
    pd.notna(dissolutions['at_some_time_within']) |
    pd.notna(dissolutions['uri']) |
    pd.notna(dissolutions['certainty_comment']) |
    pd.notna(dissolutions['date_complement']) |
    pd.notna(dissolutions['date_note'])
].reset_index(drop=True)

# a.infos(dissolutions)

### 4.6/ Generate CSV for validation

In [121]:
dissolutions.to_csv('./dissolutions.csv', index=False)

a.infos(dissolutions)

Shape:  (6850, 12) - extract:


,pk_coac,pk_group,uri,begin_of_the_begin,begin_of_the_end,end_of_the_begin,end_of_the_end,ongoing_throughout,at_some_time_within,certainty_comment,date_complement,date_note
0,CoAc10035,8366787,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"(1968, <NA>, <NA>)",<NA>,<NA>,<NA>
1,CoAc10041,8366582,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"(1968, <NA>, <NA>)",<NA>,<NA>,<NA>
2,CoAc10122,8367031,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"(1976, <NA>, <NA>)",<NA>,<NA>,<NA>
3,CoAc10126,8376521,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"(1954, <NA>, <NA>)",<NA>,<NA>,<NA>
4,CoAc10132,8366972,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,"(1987, <NA>, <NA>)",<NA>,<NA>,<NA>


### 4.7/ Date validation

In [122]:
dates_prop = ['begin_of_the_begin', 'begin_of_the_begin', 'end_of_the_begin', 'end_of_the_end', 'ongoing_throughout', 'at_some_time_within']
min_year = float('inf')
max_year = float('-inf')
for i, row in dissolutions.iterrows():
    for prop in dates_prop:
        if pd.isna(row[prop]): continue

        year = row[prop][0]
        month = row[prop][1]
        day = row[prop][2]
        
        is_pb = False

        # Year checkings
        if pd.notna(year):
            if year < min_year: min_year = year
            if year > max_year: max_year = year

        # Month checkings
        if pd.notna(month):
            if month != round(month): is_pb = True
            if month < 1 or month > 12: is_pb = True

        # Day checkings
        if pd.notna(day):    
            if day != round(day): is_pb = True
            if day < 1 or day > 31: is_pb = True

        if is_pb:
            print(row['pk_coac'], prop, row[prop], year, month, day)

print('Year range:', min_year, '-', max_year)

Year range: 700 - 2016


## 5/ Import new data

If both CSVs has been validated, import

In [123]:
# Connect to Geovistory database
db.connect_geovistory(env, pk_project, execute)
db.set_metadata({'import-id': datetime.datetime.today().strftime('%Y%m%d') + '-' + metadata_str})
db.set_insert_manner(import_manner)

[DB] Connecting to PRODUCTION Database ... Connected!


In [124]:
def get_duration(date):
    if pd.notna(date[0]) and pd.isna(date[1]) and pd.isna(date[2]): return '1 year'
    if pd.notna(date[0]) and pd.notna(date[1]) and pd.isna(date[2]): return '1 month'
    if pd.notna(date[0]) and pd.notna(date[1]) and pd.notna(date[2]): return '1 day'
    return pd.NA

### 5.1/ Create formations / dissolutions

In [125]:
formations["pk_formation"] = db.resources.create(pks.classes.formation, len(formations))
dissolutions["pk_dissolution"] = db.resources.create(pks.classes.dissolution, len(dissolutions))

Creating 10833 resources of class [60] ... Done in [00h00m03s]
Creating info_proj_rel of 10833 entities with project <6857901> ... Done in [00h00m05s]
Creating 6850 resources of class [62] ... Done in [00h00m02s]
Creating info_proj_rel of 6850 entities with project <6857901> ... Done in [00h00m03s]


### 5.2/ Link formations / dissolutions to their group

In [126]:
db.statements.create(formations['pk_formation'], pks.properties.formation_hasFormed_group, formations['pk_group'])
db.statements.create(dissolutions['pk_dissolution'], pks.properties.dissolution_dissolved_group, dissolutions['pk_group'])

Creating 10833 statements ... Updating metadata ... Done in [00h00m11s]
Creating info_proj_rel of 10833 entities with project <6857901> ... Done in [00h00m05s]
Creating 6850 statements ... Updating metadata ... Done in [00h00m06s]
Creating info_proj_rel of 6850 entities with project <6857901> ... Done in [00h00m03s]


### 5.3/ Begin of the begin

In [127]:
selection = formations[pd.notna(formations['begin_of_the_begin'])]
durations = [get_duration(date) for date in selection['begin_of_the_begin']]
time_prims = db.time_primitives.create(selection['begin_of_the_begin'], durations)
db.statements.create(selection['pk_formation'], pks.properties.timespan_beginOfTheBegin_timePrim, time_prims)

selection = dissolutions[pd.notna(dissolutions['begin_of_the_begin'])]
durations = [get_duration(date) for date in selection['begin_of_the_begin']]
time_prims = db.time_primitives.create(selection['begin_of_the_begin'], durations)
db.statements.create(selection['pk_dissolution'], pks.properties.timespan_beginOfTheBegin_timePrim, time_prims)

Creating 491 time primitives ... Done in [00h00m00s]
Creating 491 statements ... Updating metadata ... Done in [00h00m00s]
Creating info_proj_rel of 491 entities with project <6857901> ... Done in [00h00m01s]
Creating 3622 time primitives ... Done in [00h00m00s]
Creating 3622 statements ... Updating metadata ... Done in [00h00m04s]
Creating info_proj_rel of 3622 entities with project <6857901> ... Done in [00h00m02s]


### 5.4/ Begin of the end

In [128]:
selection = formations[pd.notna(formations['begin_of_the_end'])]
durations = [get_duration(date) for date in selection['begin_of_the_end']]
time_prims = db.time_primitives.create(selection['begin_of_the_end'], durations)
db.statements.create(selection['pk_formation'], pks.properties.timespan_beginOfTheEnd_timePrim, time_prims)

selection = dissolutions[pd.notna(dissolutions['begin_of_the_end'])]
durations = [get_duration(date) for date in selection['begin_of_the_end']]
time_prims = db.time_primitives.create(selection['begin_of_the_end'], durations)
db.statements.create(selection['pk_dissolution'], pks.properties.timespan_beginOfTheEnd_timePrim, time_prims)

### 5.5/ End of the begin

In [129]:
selection = formations[pd.notna(formations['end_of_the_begin'])]
durations = [get_duration(date) for date in selection['end_of_the_begin']]
time_prims = db.time_primitives.create(selection['end_of_the_begin'], durations)
db.statements.create(selection['pk_formation'], pks.properties.timespan_endOfTheBegin_timePrim, time_prims)

selection = dissolutions[pd.notna(dissolutions['end_of_the_begin'])]
durations = [get_duration(date) for date in selection['end_of_the_begin']]
time_prims = db.time_primitives.create(selection['end_of_the_begin'], durations)
db.statements.create(selection['pk_dissolution'], pks.properties.timespan_endOfTheBegin_timePrim, time_prims)

### 5.6/ End of the end

In [130]:
selection = formations[pd.notna(formations['end_of_the_end'])]
durations = [get_duration(date) for date in selection['end_of_the_end']]
time_prims = db.time_primitives.create(selection['end_of_the_end'], durations)
db.statements.create(selection['pk_formation'], pks.properties.timespan_endOfTheEnd_timePrim, time_prims)

selection = dissolutions[pd.notna(dissolutions['end_of_the_end'])]
durations = [get_duration(date) for date in selection['end_of_the_end']]
time_prims = db.time_primitives.create(selection['end_of_the_end'], durations)
db.statements.create(selection['pk_dissolution'], pks.properties.timespan_endOfTheEnd_timePrim, time_prims)

Creating 424 time primitives ... Done in [00h00m00s]
Creating 424 statements ... Updating metadata ... Done in [00h00m01s]
Creating info_proj_rel of 424 entities with project <6857901> ... Done in [00h00m00s]
Creating 122 time primitives ... Done in [00h00m00s]
Creating 122 statements ... Updating metadata ... Done in [00h00m01s]
Creating info_proj_rel of 122 entities with project <6857901> ... Done in [00h00m00s]


### 5.7/ Ongoing throughout

In [131]:
selection = formations[pd.notna(formations['ongoing_throughout'])]
durations = [get_duration(date) for date in selection['ongoing_throughout']]
time_prims = db.time_primitives.create(selection['ongoing_throughout'], durations)
db.statements.create(selection['pk_formation'], pks.properties.timeSpan_ongoingThroughout_timePrimitive, time_prims)

selection = dissolutions[pd.notna(dissolutions['ongoing_throughout'])]
durations = [get_duration(date) for date in selection['ongoing_throughout']]
time_prims = db.time_primitives.create(selection['ongoing_throughout'], durations)
db.statements.create(selection['pk_dissolution'], pks.properties.timeSpan_ongoingThroughout_timePrimitive, time_prims)

Creating 288 time primitives ... Done in [00h00m00s]
Creating 288 statements ... Updating metadata ... Done in [00h00m00s]
Creating info_proj_rel of 288 entities with project <6857901> ... Done in [00h00m01s]
Creating 115 time primitives ... Done in [00h00m00s]
Creating 115 statements ... Updating metadata ... Done in [00h00m00s]
Creating info_proj_rel of 115 entities with project <6857901> ... Done in [00h00m00s]


### 5.8/ At some time within

In [132]:
selection = formations[pd.notna(formations['at_some_time_within'])]
durations = [get_duration(date) for date in selection['at_some_time_within']]
time_prims = db.time_primitives.create(selection['at_some_time_within'], durations)
db.statements.create(selection['pk_formation'], pks.properties.timeSpan_atSomeTimeWithin_timePrimitive, time_prims)

selection = dissolutions[pd.notna(dissolutions['at_some_time_within'])]
durations = [get_duration(date) for date in selection['at_some_time_within']]
time_prims = db.time_primitives.create(selection['at_some_time_within'], durations)
db.statements.create(selection['pk_dissolution'], pks.properties.timeSpan_atSomeTimeWithin_timePrimitive, time_prims)

Creating 9765 time primitives ... Done in [00h00m01s]
Creating 9765 statements ... Updating metadata ... Done in [00h00m10s]
Creating info_proj_rel of 9765 entities with project <6857901> ... Done in [00h00m04s]
Creating 2988 time primitives ... Done in [00h00m00s]
Creating 2988 statements ... Updating metadata ... Done in [00h00m04s]
Creating info_proj_rel of 2988 entities with project <6857901> ... Done in [00h00m01s]


### 5.9/ URI

In [133]:
selection = formations[pd.notna(formations['uri'])]
db.shortcuts.add_uris(selection['pk_formation'], selection['uri'])

selection = dissolutions[pd.notna(dissolutions['uri'])]
db.shortcuts.add_uris(selection['pk_dissolution'], selection['uri'])

Creating 6855 resources of class [967] ... Done in [00h00m02s]
Creating info_proj_rel of 6855 entities with project <6857901> ... Done in [00h00m03s]
Creating 6855 appellations ... Done in [00h00m07s]
Creating 6855 statements ... Updating metadata ... Done in [00h00m07s]
Creating info_proj_rel of 6855 entities with project <6857901> ... Done in [00h00m04s]
Creating 6855 statements ... Updating metadata ... Done in [00h00m08s]
Creating info_proj_rel of 6855 entities with project <6857901> ... Done in [00h00m03s]
Creating 48 resources of class [967] ... Done in [00h00m01s]
Creating info_proj_rel of 48 entities with project <6857901> ... Done in [00h00m00s]
Creating 48 appellations ... Done in [00h00m00s]
Creating 48 statements ... Updating metadata ... Done in [00h00m00s]
Creating info_proj_rel of 48 entities with project <6857901> ... Done in [00h00m00s]
Creating 48 statements ... Updating metadata ... Done in [00h00m00s]
Creating info_proj_rel of 48 entities with project <6857901> ... 

### 5.10/ Certainty comment

In [134]:
selection = formations[pd.notna(formations['certainty_comment'])]
pk_comments = db.resources.create(pks.classes.comment, len(selection))
pk_appes = db.appellations.create(selection['certainty_comment'])
db.statements.create(pk_comments, pks.properties.appe_hasValue_string, pk_appes)
db.statements.create(pk_comments, pks.properties.comment_hasCommentType_CommentType, 7953586)
db.statements.create(selection['pk_formation'], pks.properties.entity_hasComment_text, pk_comments)

selection = dissolutions[pd.notna(dissolutions['certainty_comment'])]
pk_comments = db.resources.create(pks.classes.comment, len(selection))
pk_appes = db.appellations.create(selection['certainty_comment'])
db.statements.create(pk_comments, pks.properties.appe_hasValue_string, pk_appes)
db.statements.create(pk_comments, pks.properties.comment_hasCommentType_CommentType, 7953586)
db.statements.create(selection['pk_dissolution'], pks.properties.entity_hasComment_text, pk_comments)

Creating 4070 resources of class [900] ... Done in [00h00m01s]
Creating info_proj_rel of 4070 entities with project <6857901> ... Done in [00h00m02s]
Creating 4070 appellations ... Done in [00h00m01s]
Creating 4070 statements ... Updating metadata ... Done in [00h00m05s]
Creating info_proj_rel of 4070 entities with project <6857901> ... Done in [00h00m02s]
Creating 4070 statements ... Updating metadata ... Done in [00h00m05s]
Creating info_proj_rel of 4070 entities with project <6857901> ... Done in [00h00m02s]
Creating 4070 statements ... Updating metadata ... Done in [00h00m04s]
Creating info_proj_rel of 4070 entities with project <6857901> ... Done in [00h00m02s]
Creating 4073 resources of class [900] ... Done in [00h00m01s]
Creating info_proj_rel of 4073 entities with project <6857901> ... Done in [00h00m02s]
Creating 4073 appellations ... Done in [00h00m02s]
Creating 4073 statements ... Updating metadata ... Done in [00h00m04s]
Creating info_proj_rel of 4073 entities with project 

### 5.11/ Date_complement

In [135]:
selection = formations[pd.notna(formations['date_complement'])]
pk_comments = db.resources.create(pks.classes.comment, len(selection))
pk_appes = db.appellations.create(selection['date_complement'])
db.statements.create(pk_comments, pks.properties.appe_hasValue_string, pk_appes)
db.statements.create(pk_comments, pks.properties.comment_hasCommentType_CommentType, 8065621)
db.statements.create(selection['pk_formation'], pks.properties.entity_hasComment_text, pk_comments)

selection = dissolutions[pd.notna(dissolutions['date_complement'])]
pk_comments = db.resources.create(pks.classes.comment, len(selection))
pk_appes = db.appellations.create(selection['date_complement'])
db.statements.create(pk_comments, pks.properties.appe_hasValue_string, pk_appes)
db.statements.create(pk_comments, pks.properties.comment_hasCommentType_CommentType, 8065621)
db.statements.create(selection['pk_dissolution'], pks.properties.entity_hasComment_text, pk_comments)

Creating 62 resources of class [900] ... Done in [00h00m00s]
Creating info_proj_rel of 62 entities with project <6857901> ... Done in [00h00m00s]
Creating 62 appellations ... Done in [00h00m00s]
Creating 62 statements ... Updating metadata ... Done in [00h00m00s]
Creating info_proj_rel of 62 entities with project <6857901> ... Done in [00h00m00s]
Creating 62 statements ... Updating metadata ... Done in [00h00m01s]
Creating info_proj_rel of 62 entities with project <6857901> ... Done in [00h00m00s]
Creating 62 statements ... Updating metadata ... Done in [00h00m00s]
Creating info_proj_rel of 62 entities with project <6857901> ... Done in [00h00m00s]


### 5.11/ Date note

In [136]:
selection = formations[pd.notna(formations['date_note'])]
pk_comments = db.resources.create(pks.classes.comment, len(selection))
pk_appes = db.appellations.create(selection['date_note'])
db.statements.create(pk_comments, pks.properties.appe_hasValue_string, pk_appes)
db.statements.create(pk_comments, pks.properties.comment_hasCommentType_CommentType, 8065632)
db.statements.create(selection['pk_formation'], pks.properties.entity_hasComment_text, pk_comments)

selection = dissolutions[pd.notna(dissolutions['date_note'])]
pk_comments = db.resources.create(pks.classes.comment, len(selection))
pk_appes = db.appellations.create(selection['date_note'])
db.statements.create(pk_comments, pks.properties.appe_hasValue_string, pk_appes)
db.statements.create(pk_comments, pks.properties.comment_hasCommentType_CommentType, 8065632)
db.statements.create(selection['pk_dissolution'], pks.properties.entity_hasComment_text, pk_comments)

Creating 43 resources of class [900] ... Done in [00h00m00s]
Creating info_proj_rel of 43 entities with project <6857901> ... Done in [00h00m00s]
Creating 43 appellations ... Done in [00h00m00s]
Creating 43 statements ... Updating metadata ... Done in [00h00m01s]
Creating info_proj_rel of 43 entities with project <6857901> ... Done in [00h00m00s]
Creating 43 statements ... Updating metadata ... Done in [00h00m00s]
Creating info_proj_rel of 43 entities with project <6857901> ... Done in [00h00m00s]
Creating 43 statements ... Updating metadata ... Done in [00h00m00s]
Creating info_proj_rel of 43 entities with project <6857901> ... Done in [00h00m00s]
Creating 1 resources of class [900] ... Done in [00h00m00s]
Creating info_proj_rel of 1 entities with project <6857901> ... Done in [00h00m01s]
Creating 1 appellations ... Done in [00h00m00s]
Creating 1 statements ... Updating metadata ... Done in [00h00m00s]
Creating info_proj_rel of 1 entities with project <6857901> ... Done in [00h00m00s]